# 0. Libraries

In [8]:
import sys
import os

# Add the parent directory (graphrag_pipeline) to the Python path (needed for importing
# modules in parent directory)
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Utilities
import asyncio
from dotenv import load_dotenv
import os
import json
from google import genai
import polars as pl
from library.kg_builder import CustomKGPipeline, build_kg_from_df
from library.kg_builder.utilities import GeminiLLM
from neo4j_graphrag.experimental.components.resolver import (
    SpaCySemanticMatchResolver, FuzzyMatchResolver, SinglePropertyExactMatchResolver
)
import tqdm.notebook as tqdm

# Neo4j and Neo4j GraphRAG imports
import neo4j
from neo4j_graphrag.embeddings import SentenceTransformerEmbeddings

Let's first check the available Gemini models.

In [2]:
load_dotenv('.env', override=True)

gemini_api_key = os.getenv('GEMINI_API_KEY')

if gemini_api_key:
    client = genai.Client(api_key=gemini_api_key)  # Configure the API key for genai
else:
    raise ValueError("GEMINI_API_KEY environment variable is not set.")

# Display available models
for model in client.models.list():
    print(model)

name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=1024 output_token_limit=1 supported_actions=['embedText', 'countTextTokens'] default_checkpoint_id=None checkpoints=None
name='models/gemini-1.0-pro-vision-latest' display_name='Gemini 1.0 Pro Vision' description='The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=12288 output_token_limit=4096 supported_actions=['generateContent', 'countTokens'] default_checkpoint_id=None checkpoints=None
name='models/gemini-pro-vision' display_name='Gemini 1.0 Pro Vi

We also have to make sure that the corresponding SpaCy model for text embedding used at the resolving step is installed.

In [3]:
import importlib.util
import subprocess
import sys

import spacy


def ensure_spacy_model(model_name):
    if importlib.util.find_spec(model_name) is None:
        print(f"Model '{model_name}' not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "spacy", "download", model_name])
    else:
        print(f"Model '{model_name}' is already installed.")

# Use it for 'en_core_web_lg'
ensure_spacy_model("en_core_web_lg")  # Model used for resolving entities in the KG pipeline

Model 'en_core_web_lg' is already installed.


In [4]:
config_files_path = os.path.join(os.path.dirname(os.getcwd()), 'config_files')

with open(os.path.join(config_files_path, 'kg_building_config_loc.json'), 'r') as f:
    config = json.load(f)

config['prompt_template_config']

{'use_default': False,
 'template': "For each row in the DataFrame, create a node with the label 'Location'. Do not create nodes for Document and Text chunk. The node should have three properties: 'name', 'admin 1', and 'country'. These should be populated with the values from the DataFrame columns 'Name', 'Admin1', and 'Country' respectively."}

# 1. Loading the data

The data is loaded here as a reference, but it is loaded again inside the pipeline below.

## 1.2. Factal sample data

In [5]:
global df1

df1 = pl.read_csv(os.path.join('sample_data', 'factal_single_topic_report-2025-05-01-2025-06-05.csv'))
df1 = df1.rename({"Associated topics": "Country"})
df1 = df1.rename({"Published text": "Text"})

df1=df1.head(20)

# Create an index for each row
df1 = df1.with_row_index(name="id", offset=1)
# Convert the "id" to a string to ensure it is treated as a document ID
df1 = df1.with_columns(pl.col('id').cast(pl.String))
    
df1.head(2)

id,Published date,Severity,Text,Translated text,Original language,Source URL,Status,Country
str,str,i64,str,str,str,str,str,str
"""1""","""2025-06-03 15:28:28.271179+00:…",3,"""WFP and UNICEF now say five me…",null,null,"""https://www.unicef.org/press-r…","""published""","""Sudan"""
"""2""","""2025-06-03 10:10:04.994458+00:…",3,"""""Multiple casualties"" after WF…",null,null,"""https://www.reuters.com/world/…","""published""","""Sudan"""


### Load Admin1 locations from HDX database

In [6]:
global df2
admin1 = pl.read_csv(r"C:\Users\matia\Downloads\global_pcodes_adm_1_2.csv")
df2 = admin1.filter(pl.col('Location') == 'SDN').clone()
df2 = df2.with_columns(pl.lit('Sudan').alias('Country'))
df2 = df2.rename({"Name": "Name"})

sudan_states_mapping = {
    "SD01": "Khartoum",
    "SD02": "North Darfur",
    "SD03": "South Darfur",
    "SD04": "West Darfur",
    "SD05": "East Darfur",
    "SD06": "Central Darfur",
    "SD07": "South Kordofan",
    "SD08": "Blue Nile",
    "SD09": "White Nile",
    "SD10": "Red Sea",
    "SD11": "Kassala",
    "SD12": "Gedaref",
    "SD13": "North Kordofan",
    "SD14": "Sennar",
    "SD15": "Aj Jazirah",
    "SD16": "River Nile",
    "SD17": "Northern",
    "SD18": "West Kordofan",
    "SD19": "Abyei PCA"
}

df2 = df2.with_columns(
    pl.when(pl.col("Parent P-Code") == "SDN")
    .then(pl.col("Name"))
    .otherwise(
        pl.col("Parent P-Code").replace(sudan_states_mapping, default=pl.col("Parent P-Code"))
    )
    .alias("Admin1")
)

df2 = df2.select(['Name', 'Admin1', 'Country'])

# Create an index for each row if df2 doesn't already have an 'id' column
if 'id' not in df2.columns:
    df2 = df2.with_row_index(name="id", offset=1)
    # Convert the "id" to a string to ensure it is treated as a document ID
    df2 = df2.with_columns(pl.col('id').cast(pl.String))
        
df2.head(2)

C:\Users\matia\AppData\Local\Temp\ipykernel_51644\1507764610.py:33: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  pl.col("Parent P-Code").replace(sudan_states_mapping, default=pl.col("Parent P-Code"))


id,Name,Admin1,Country
str,str,str,str
"""1""","""Khartoum""","""Khartoum""","""Sudan"""
"""2""","""North Darfur""","""North Darfur""","""Sudan"""


In [ ]:
# Try to convert using Polars' built-in method
try:
	pandas_df2 = df2.to_pandas()
except ImportError:
	# If PyArrow is not available, convert manually 
	# by using the dict-based conversion which doesn't require PyArrow
	pandas_df2 = pd.DataFrame({col: df2[col].to_list() for col in df2.columns})

# Display the first few rows of the pandas DataFrame
print(pandas_df2.head())

# 2. Running the pipeline

## 2.2. With a data frame

### A. Using the `SpaCySemanticMatchResolver`

More useful information about the resolvers can be found in the [user guide](https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_kg_builder.html#entity-resolver). Below, we use different resolvers (from the most aggressive - spaCy to the most conservative - exact matching) to get a broad overview of the performance results.

### Embed Locations from list of Admin units

In [10]:
async def embed_locations(df):
    
    # Find path to config_files folder
    config_files_path = os.path.join(os.path.dirname(os.getcwd()), 'config_files')

    # Load environment variables from .env file
    load_dotenv(os.path.join(config_files_path, '.env'), override=True)
    
    with open(os.path.join(config_files_path, 'kg_building_config_loc.json'), 'r') as f:
        config = json.load(f)
        
    neo4j_uri = os.getenv('NEO4J_URI')
    neo4j_username = os.getenv('NEO4J_USERNAME')
    neo4j_password = os.getenv('NEO4J_PASSWORD')
    gemini_api_key = os.getenv('GEMINI_API_KEY')
    
    # Create Neo4j driver
    driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
    
    # Keep track of processed nodes
    processed_nodes = 0

    def create_location_nodes(tx, name, admin1, country):
        query = """
        MERGE (l:Location {name: $name, `admin 1`: $admin1, country: $country})
        """
        tx.run(query, name=name, admin1=admin1, country=country)
        return 1  # Return count of node processed

    # Main logic to insert nodes from the DataFrame
    try:
        with driver.session() as session:
            # Add progress bar
            for row in tqdm.tqdm(df.iter_rows(named=True), total=len(df), desc="Creating location nodes"):
                # Process each row and count it
                processed_nodes += session.execute_write(
                    create_location_nodes, 
                    row["Name"], 
                    row["Admin1"], 
                    row["Country"]
                )
    finally:
        driver.close()
    
    # Return the count of processed nodes
    return processed_nodes

# Run the main function
results = await embed_locations(df2)
print(f"Processed {results} nodes")

Creating location nodes:   0%|          | 0/208 [00:00<?, ?it/s]

Processed 208 nodes


#### With Factal

In [9]:
# Example usage code
async def main():

    global df1
    
    # Find path to config_files folder
    config_files_path = os.path.join(os.path.dirname(os.getcwd()), 'config_files')

    # Load environment variables from .env file
    load_dotenv(os.path.join(config_files_path, '.env'), override=True)
    
    with open(os.path.join(config_files_path, 'kg_building_config2.json'), 'r') as f:
        config = json.load(f)
    
    # Neo4j connection
    neo4j_uri = os.getenv('NEO4J_URI')
    neo4j_username = os.getenv('NEO4J_USERNAME')
    neo4j_password = os.getenv('NEO4J_PASSWORD')
    gemini_api_key = os.getenv('GEMINI_API_KEY')
    
    # Check if gemini_api_key is set
    if gemini_api_key:
        pass
    else:
        raise ValueError("Gemini API key is not set. Please provide a valid API key.")

    # Initialize LLM
    llm = GeminiLLM(
        model_name=config['llm_config']['model_name'],
        google_api_key=gemini_api_key,
        model_params=config['llm_config']['model_params']
    )
    
    # Initialize embedder
    embedder = SentenceTransformerEmbeddings(model=config['embedder_config']['model_name'])
    
    # Configure text splitter
    text_splitter_config = config['text_splitter_config'] 
    
    # Create the pipeline - use with statement to ensure proper resource management
    # and to ensure the driver is closed after use
    all_results = []
    with neo4j.GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password)) as driver:
        
        # Initialize entity resolver
        resolver = SpaCySemanticMatchResolver(  # Merge nodes with same label and similar textual properties
            driver,
            filter_query=None,  # "WHERE (entity)-[:FROM_CHUNK]->(:Chunk)-[:FROM_DOCUMENT]->(doc:Document {id = 'docId'}",  # Used to reduce the resolution scope to a specific document
            resolve_properties=["name"],  # Properties to use for resolution (default is "name")
            similarity_threshold=0.8,  # The similarity threshold above which nodes are merged (default is 0.8). Higher threshold will result in less false positives, but may miss some matches. 
            spacy_model="en_core_web_lg"  # spaCy model to use for resolution (default is "en_core_web_lg")
        )

        # Initialize the custom KG pipeline
        kg_pipeline = CustomKGPipeline(
            llm=llm,
            driver=driver,
            embedder=embedder,
            schema_config=config['schema_config'],
            prompt_template=config['prompt_template_config']['template'] if config['prompt_template_config'].get('use_default') == False else None,
            text_splitter_config=text_splitter_config,
            resolver=resolver,
            examples_config=None,  # Use None if no examples are provided
            on_error='RAISE',
            batch_size=1000,
            max_concurrency=5
        )
        
        # Define metadata mapping (document properties additional to base field 
        # to dataframe columns)
        metadata_mapping1 = {
            "source": "Source URL",
            "published_date": "Published date",
            "country": "Country"
        }
        
        # metadata_mapping2 = {
        #     "name": "Name",
        #     "admin 1": "Admin1",
        #     "country": "Country"
        # }
        
        # Process the First dataframe
        print("Processing the first dataframe...")
        results_df1 = await build_kg_from_df(
            kg_pipeline=kg_pipeline,
            df=df1,
            document_base_field='id',
            text_column='Text',
            document_metadata_mapping=metadata_mapping1,
            document_id_column=None  # Use default document ID generation
        )
        all_results.extend(results_df1)

    #     # Process the Second dataframe
    #     print("Processing the second dataframe...")
    #     results_df2 = await build_kg_from_df(
    #         kg_pipeline=kg_pipeline,
    #         df=df2,
    #         document_base_field='id',
    #         text_column='Name',
    #         document_metadata_mapping=metadata_mapping2,
    #         document_id_column=None  # Use default document ID generation
    #     )
    #     all_results.extend(results_df2)
    
    return all_results

# Asyncio event loop to run the main function in a Jupyter notebook
all_results = await main()
print(f"Processed {len(all_results)} documents")

# # Asyncio event loop to run the main function in a script
# if __name__ == "__main__":
#     results = asyncio.run(main())
#     print(f"Processed {len(results)} documents")

Processing the first dataframe...
Processing row 1 of 20


LLM response has improper format for chunk_index=0


Result: run_id='197dd09d-8bc2-4d81-8d1d-ead100f418e6' result={'resolver': {'number_of_nodes_to_resolve': 0, 'number_of_created_nodes': 0}}
Elapsed time: 3.79 seconds
Estimated time remaining: 71.98 seconds

Processing row 2 of 20


LLM response has improper format for chunk_index=0


CancelledError: 

# Entity Resolution Pipeline

In [ ]:
# Connect to Neo4j
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=("neo4j", os.getenv("NEO4J_PASSWORD"))
)

# Load embedding model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def get_embedding(text):
    return model.encode(text).tolist()

# List of relevant node labels for deduplication
ENTITY_LABELS = ["Event", "Actor", "Location"]

def get_all_entities():
    query_template = """
    MATCH (n:{label})
    RETURN elementId(n) AS id, n.name AS name, labels(n) AS labels, properties(n) AS properties
    """
    all_entities = []
    with driver.session() as session:
        for label in ENTITY_LABELS:
            result = session.run(query_template.format(label=label)).data()
            all_entities.extend(result)
    return all_entities

def cosine_similarity(vec1, vec2):
    if vec1 is None or vec2 is None or len(vec1) == 0 or len(vec2) == 0:
        return 0  # If either vector is None or empty, return 0 similarity
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot / (norm1 * norm2) if norm1 > 0 and norm2 > 0 else 0

def find_similar_entities(threshold=0.7):
    entities = get_all_entities()
    print(f"Found {len(entities)} entities to process")
    
    # Compute embeddings
    for entity in entities:
        # Filter out "__KGBuilder__" from labels
        filtered_labels = [l for l in entity['labels'] if l != "__KGBuilder__" and l != "__Entity__"]
        if filtered_labels:  # If there are any labels left
            entity['primary_label'] = filtered_labels[0]
        else:
            entity['primary_label'] = entity['labels'][0]  # Fallback if only __KGBuilder__ is present
            
        # Start with entity label/type and name
        text = f"Type: {entity['primary_label']}\nName: {entity['name']}\n"
        
        # Rest of embedding code remains the same
        for key, value in entity['properties'].items():
            if key != 'embedding' and value is not None:
                if isinstance(value, (list, dict)):
                    formatted_value = str(value)
                else:
                    formatted_value = value
                text += f"{key}: {formatted_value}\n"
        
        entity['embedding'] = get_embedding(text)
     
    # Find similar pairs with the new label comparison
    similar_pairs = []
    total_comparisons = sum(range(len(entities)))
    for i, e1 in enumerate(entities):
        for j, e2 in enumerate(entities[i + 1:], i + 1):
            # Only compare if they have the same primary label (excluding __KGBuilder__)
            if e1['primary_label'] != e2['primary_label'] or e1['primary_label'] == "__KGBuilder__":
                continue
            
            sim = cosine_similarity(e1['embedding'], e2['embedding'])
            if sim > threshold:
                similar_pairs.append({
                    "id1": e1['id'],
                    "id2": e2['id'],
                    "name1": e1['name'],
                    "name2": e2['name'],
                    "type1": e1['primary_label'],
                    "type2": e2['primary_label'],
                    "similarity": sim
                })
    
    # Create SAME_AS relationships
    query = """
    MATCH (a), (b)
    WHERE elementId(a) = $id1 AND elementId(b) = $id2
    MERGE (a)-[:SAME_AS {similarity: $similarity}]->(b)
    """
    with driver.session() as session:
        for pair in tqdm.tqdm(similar_pairs, desc="Computing similarity scores"):
            session.run(query, pair)
    
    return similar_pairs

def check_apoc():
    try:
        with driver.session() as session:
            session.run("CALL apoc.help('create')")
            print("APOC is available.")
            return True
    except Exception as e:
        print(f"APOC not available: {e}")
        return False

# === MAIN EXECUTION ===
check_apoc()
pairs = find_similar_entities()
pairs = sorted(pairs, key=lambda x: x["similarity"], reverse=True)
print(f"Found {len(pairs)} similar entity pairs.")

APOC is available.
Found 206 entities to process


Computing similarity scores:   0%|          | 0/20805 [00:00<?, ?it/s]

BufferError: Existing exports of data: object cannot be re-sized

In [ ]:
print(pairs)

In [ ]:
def merge_similar_nodes():
    merge_query = """
    // Process one pair of nodes at a time to avoid conflicts
    MATCH (n1)-[r:SAME_AS]->(n2)
    WHERE n1 IS NOT NULL AND n2 IS NOT NULL
    
    // Copy properties from n2 to n1 if they don't exist in n1
    WITH n1, n2, [key IN keys(n2) WHERE NOT key IN keys(n1)] AS newKeys
    FOREACH (key IN newKeys | SET n1[key] = n2[key])
    
    // Get all outgoing relationships from n2 (except SAME_AS)
    WITH n1, n2
    OPTIONAL MATCH (n2)-[outRel]->(target)
    WHERE target IS NOT NULL AND type(outRel) <> 'SAME_AS'
    
    // Create equivalent relationships from n1 if they don't already exist
    WITH n1, n2, outRel, target, type(outRel) AS relType
    WHERE NOT EXISTS((n1)-[:`${relType}`]->(target))
    
    // Create new relationship with the same properties
    FOREACH (_ IN CASE WHEN outRel IS NOT NULL THEN [1] ELSE [] END |
        CREATE (n1)-[newRel:`${relType}`]->(target)
        SET newRel = properties(outRel)
    )
    
    // Return the node pair for the next phase
    WITH DISTINCT n1, n2
    
    // Handle incoming relationships
    OPTIONAL MATCH (source)-[inRel]->(n2)
    WHERE source IS NOT NULL AND source <> n1 AND type(inRel) <> 'SAME_AS'
    
    // Create equivalent relationships to n1 if they don't already exist
    WITH n1, n2, inRel, source, type(inRel) AS relType
    WHERE NOT EXISTS((source)-[:`${relType}`]->(n1))
    
    // Create new relationship with the same properties
    FOREACH (_ IN CASE WHEN inRel IS NOT NULL THEN [1] ELSE [] END |
        CREATE (source)-[newRel:`${relType}`]->(n1)
        SET newRel = properties(inRel)
    )
    
    // Return distinct pairs for deletion phase
    WITH DISTINCT n1, n2
    
    // Delete the second node and all its relationships
    DETACH DELETE n2
    
    RETURN count(n2) AS mergedCount
    """
    
    try:
        with driver.session() as session:
            result = session.run(merge_query)
            record = result.single()
            return record["mergedCount"] if record else 0
    except Exception as e:
        print(f"Error during node merging: {e}")
        return 0

merged = merge_similar_nodes()
print(f"Merged {merged} nodes.")

### Node similarity

In [ ]:
node_similarity_query = """
CALL gds.nodeSimilarity.stream('amazonGraph')
YIELD node1, node2, similarity as node_similarity
WHERE 'Company' IN labels(gds.util.asNode(node1)) AND 'Company' IN labels(gds.util.asNode(node2))
AND node_similarity < 1
RETURN gds.util.asNode(node1).name AS Company1, gds.util.asNode(node2).name AS Company2, node_similarity
ORDER BY node_similarity DESCENDING, Company1, Company2
"""

def results_to_df(query: str) -> pd.DataFrame:
    results = gds.execute_query(query)[0]
    df = pd.DataFrame(results, columns=results[0].keys())
    return df

df_node_similarity = results_to_df(node_similarity_query)
print(df_node_similarity)

### Create SAME_AS relationships

In [ ]:
def create_same_as_relationship(df, column_name):
    # Iterate over the DataFrame rows
    for _, row in df.iterrows():
        node1 = row[column_name + '1']
        node2 = row[column_name + '2']

        # Run Cypher query to create 'SAME_AS' relationship
        score = row["combined_score"]
        if score > 0.20:
            query = f"MATCH (n1), (n2) WHERE n1.name = '{node1}' AND n2.name = '{node2}' CREATE (n1)-[:SAME_AS]->(n2)"
            gds.execute_query(query)

create_same_as_relationship(selected_df, "Company")

### Merge nodes with SAME_AS relationship

In [ ]:
merge_query = """
MATCH (n1)-[r:SAME_AS]->(n2)
WITH n1, n2, collect(r) as relsToDel

FOREACH (rel IN relsToDel | DELETE rel)
WITH collect(DISTINCT n1) + collect(DISTINCT n2) AS nodesToMerge

UNWIND nodesToMerge AS node

WITH collect(DISTINCT node) AS uniqueNodesToMerge
CALL apoc.refactor.mergeNodes(uniqueNodesToMerge, {mergeRels:true}) YIELD node
RETURN node
"""

gds.execute_query(merge_query)